In [1]:
import torch
import torch.nn as nn
import data.datasets as dataSource
import torch.utils.data as tdata
import numpy as np
import scipy.constants as _scipy_constants

import cvxpy as cp

In [7]:
dataset_train, dataset_val = dataSource.Dipole_H_train, dataSource.Dipole_H_val
#print(dataset_train.input_columns)
batch_size = 64
important_cols = ["B_design","aper_x", "aper_y", "fieldTolerance", "maxCurrentDensity" , "rho0", "w_leg_factor", "usedPowerInPercent"]
important_cols_order = sorted(important_cols,key=lambda c: dataset_train.input_columns.index(c))

print(important_cols)
print(important_cols_order)

input_mask = [True if col in important_cols else False for col in dataset_train.input_columns]
train_loader, val_loader = tdata.DataLoader(dataset_train,batch_size), tdata.DataLoader(dataset_val,batch_size)

target_index_dict = {col:i for i,col in enumerate(dataset_train.target_columns)}
target_cols = ["B0","gfr_x_1e-3","gfr_y_1e-3"]
output_mask = [True if col in target_cols else False for col in dataset_train.target_columns]

['B_design', 'aper_x', 'aper_y', 'fieldTolerance', 'maxCurrentDensity', 'rho0', 'w_leg_factor', 'usedPowerInPercent']
['fieldTolerance', 'maxCurrentDensity', 'rho0', 'usedPowerInPercent', 'w_leg_factor', 'B_design', 'aper_x', 'aper_y']


In [41]:
oid = {key : important_cols.index(key) for key in important_cols_order}

def get_dipole_continuous(x: torch.Tensor):
    # x.shape = (b,7) with batchDim b and 6 "actual" inputs 'important cols'
    aper_x = x[:,oid["aper_x"]]
    aper_y = x[:,oid["aper_y"]]
    maxCurrentDensity = x[:,oid["maxCurrentDensity"]]
    fieldTolerance = x[:,oid["fieldTolerance"]]
    B_design = x[:,oid["B_design"]]
    rho0 = x[:,oid["rho0"]]
    w_leg_factor = x[:,oid["w_leg_factor"]]
    usedPowerInPercent = x[:,oid["usedPowerInPercent"]]

    xnorm_unoptimised = -0.36 * torch.log(fieldTolerance) - 0.90
    xnorm_optimised = -0.14 * torch.log(fieldTolerance) - 0.25
    gfr_x = aper_x * rho0
    gfr_y = aper_y * rho0
    dipole_corner_norm_0 = rho0**2+xnorm_unoptimised
    polecorner_x = aper_x + xnorm_unoptimised * aper_y

    w = 2*polecorner_x + 2 * polecorner_x * 0.05
    w_leg = 0.5 * (w + 2 * aper_y) * B_design / 2.15 * w_leg_factor

    totalCurrentDesign = B_design * aper_y / (2 * _scipy_constants.mu_0)
    totalCurrent = usedPowerInPercent * 0.01 * totalCurrentDesign

    A_min = totalCurrentDesign / 1e6 / (maxCurrentDensity * (75 * 0.01))
    coil_width = (A_min *2) ** 0.5
    coil_height = coil_width * 0.5
    
    stack = torch.cat([
        x,
        gfr_x.unsqueeze(1),
        gfr_y.unsqueeze(1),
        xnorm_unoptimised.unsqueeze(1),
        xnorm_optimised.unsqueeze(1),
        dipole_corner_norm_0.unsqueeze(1),
        polecorner_x.unsqueeze(1),
        w.unsqueeze(1),
        w_leg.unsqueeze(1),
        totalCurrentDesign.unsqueeze(1),
        totalCurrent.unsqueeze(1),
        coil_width.unsqueeze(1),
        coil_height.unsqueeze(1),
        A_min.unsqueeze(1),
        ],dim=1)
    return stack


In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class MyRegNet(nn.Module):
    def __init__(self,h_size=32):
        super().__init__()
        self.bn0 = nn.BatchNorm1d(len(important_cols)+13, dtype=torch.double)
        self.fc1 = nn.Linear(len(important_cols)+13,h_size, dtype=torch.double)
        self.nl1 = nn.LeakyReLU()
        self.bn1 = nn.BatchNorm1d(h_size, dtype=torch.double)
        self.do1 = nn.Dropout1d()
        self.fc2 = nn.Linear(h_size,h_size, dtype=torch.double)
        self.nl2 = nn.LeakyReLU()
        self.bn2 = nn.BatchNorm1d(h_size, dtype=torch.double)
        self.do2 = nn.Dropout1d()
        self.fc3 = nn.Linear(h_size,3, dtype=torch.double)
    def forward(self,x):
        x = get_dipole_continuous(x)
        x = self.bn0(x)
        x = self.fc1(x)
        x = self.nl1(x)
        x = self.bn1(x)
        #x = self.do1(x)
        x = self.fc2(x)
        x = self.nl2(x)
        x = self.bn2(x)
        #x = self.do2(x)
        x = self.fc3(x)
        return x

network = MyRegNet().to(device)

In [74]:
loss_mask_df = dataset_train.dataframe[["B0","gfr_x_1e-3","gfr_y_1e-3"]].mean()

In [75]:
n_epochs = 200
lr = 5e-5
optim = torch.optim.Adam(network.parameters(), lr)
loss_fun = nn.MSELoss()

for epoch in range(n_epochs):
    network.train()
    loss_list = []
    for x,y in train_loader:
        selected_x = x[:,input_mask].double().to(device)
        selected_y = y[:,output_mask].double().to(device)
        pred = network(selected_x)

        #loss_mask = (torch.tensor(list(loss_mask_df))* torch.tensor([10,1,1])).repeat((x.shape[0],1)).to(device)
        loss = loss_fun(pred,selected_y)
        loss_list.append(loss.detach().item())
        optim.zero_grad()
        loss.backward()
        optim.step()
    epoch_train_loss = torch.tensor(loss_list).mean()

    network.eval()
    loss_list = []
    for x,y in val_loader:
        selected_x = x[:,input_mask].double().to(device)
        selected_y = y[:,output_mask].double().to(device)
        pred = network(selected_x)

        #loss_mask = (torch.tensor(list(loss_mask_df))* torch.tensor([10,1,1])).repeat((x.shape[0],1)).to(device)
        loss = loss_fun(pred ,selected_y )
        loss_list.append(loss.detach().item())
    epoch_val_loss = torch.tensor(loss_list).mean()
    print(epoch,epoch_train_loss,epoch_val_loss,end="\r")


KeyboardInterrupt: 

In [64]:
torch.save(network.state_dict(), "network_design_params")

In [76]:

network = MyRegNet().to(device)
network.load_state_dict(torch.load("network_design_params"))

/tmp/ipykernel_1110960/1750107852.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  network.load_state_dict(torch.load("network_design_params"))


<All keys matched successfully>

In [80]:
errors = {error_type: {"B0":[],"gfr_x":[],"gfr_y":[]} for error_type in ["abs","rel"]}

single_sample_loader =  tdata.DataLoader(dataset_val, 1)

network.eval()
for x,y in single_sample_loader:
    
    selected_x = x[:,input_mask].double().to(device)
    selected_y = y[:,output_mask].double().to(device)
    pred = network(selected_x)
    #print(pred,selected_y)
    abs_errors = torch.abs(pred - selected_y)
    errors["abs"]["B0"].append(abs_errors[:,0].detach().item())
    errors["abs"]["gfr_x"].append(abs_errors[:,1].detach().item())
    errors["abs"]["gfr_y"].append(abs_errors[:,2].detach().item())

    rel_errors = torch.abs(1 - (pred/selected_y))
    errors["rel"]["B0"].append(rel_errors[:,0].detach().item())
    errors["rel"]["gfr_x"].append(rel_errors[:,1].detach().item())
    errors["rel"]["gfr_y"].append(rel_errors[:,2].detach().item())
mean_errors = {error_type: {quantity: sum(v)/len(v) for quantity, v in dictt.items()} for error_type, dictt in errors.items() }
print(mean_errors)

{'abs': {'B0': 0.03093236719500123, 'gfr_x': 0.014907938658596584, 'gfr_y': 0.005870515391147503}, 'rel': {'B0': 0.08968141834874543, 'gfr_x': 0.13212382808958228, 'gfr_y': 0.17390795346346155}}


In [93]:
A = []
b = []

def onehot(name, neg):
    out = [0] * len(important_cols_order)
    out[important_cols_order.index(name)] = -1 if neg else 1
    return out

for name in important_cols_order:
    # constraints for minima: x > a --> -x < -a
    A.append(onehot(name,True))
    b.append(-dataset_train.input_mins[name])
    
    # constraints for maxima: x < a
    if name == "usedPowerInPercent":
        A.append(onehot(name,False))
        b.append(100)
    else:
        A.append(onehot(name,False))
        b.append(dataset_train.input_maxs[name])

In [82]:
A_rel = []
b_rel = []

#['fieldTolerance', 'maxCurrentDensity', 'rho0', 'usedPowerInPercent', 'w_leg_factor', 'B_design', 'aper_x', 'aper_y']

# aper_x > aper_y --> aper_y - aper_x < 0
A_rel.append([0, 0, 0, 0, 0, 0, -1, 1])
b_rel.append(0)

# aper_x < 5 * aper_y --> aper_x - 5 * aper_y < 0
A_rel.append([0, 0, 0, 0, 0, 0, 1, -3])
b_rel.append(0)



A = A + A_rel
b = b + b_rel

In [84]:
# ratio constraints 
# compute min and max ratio for every pair of inputs and ensure the ratio is in that range with two linear constraints
# e.g. min < a/b < max -->  min * b - a < 0 and a - max * b < 0
# with slack. min * (1+e) < a/b < max * (1-e) -->  (min (1+ e)) * b - a < 0 and a - (max*(1-e)) * b < 0

A_rat = []
b_rat = []
epsilon = 1e-5

for i in range(len(important_cols_order)-1):
    for j in range(i+1,len(important_cols_order)):
        quantity_1 = important_cols_order[i]
        quantity_2 = important_cols_order[j]

        # ratio = column i/ colimn j
        ratio_df = dataset_train.dataframe.assign(ratio=dataset_train.dataframe.apply(lambda r: r[quantity_1] / r[quantity_2], axis=1))

        minimum = ratio_df["ratio"].min() 
        maximum = ratio_df["ratio"].max()

        minimum *= 1+epsilon
        maximum *= 1-epsilon

        # min * j - i < 0
        min_row = [0]*len(important_cols)
        min_row[i] = -1
        min_row[j] = minimum
        A_rat.append(min_row)
        b_rat.append(0)

        #i - max * k < 0
        max_row = [0]*len(important_cols)
        max_row[i] = 1
        max_row[j] = -maximum
        A_rat.append(max_row)
        b_rat.append(0)

        print(quantity_1,quantity_2,minimum,maximum, "-->", min_row,"< 0 and", max_row,"< 0")


A = A + A_rat
b = b + b_rat

fieldTolerance maxCurrentDensity 1.0000100000000001e-05 0.0199998 --> [-1, 1.0000100000000001e-05, 0, 0, 0, 0, 0, 0] < 0 and [1, -0.0199998, 0, 0, 0, 0, 0, 0] < 0
fieldTolerance rho0 0.00010000100000000001 0.0199998 --> [-1, 0, 0.00010000100000000001, 0, 0, 0, 0, 0] < 0 and [1, 0, -0.0199998, 0, 0, 0, 0, 0] < 0
fieldTolerance usedPowerInPercent 8.000080000000001e-07 0.0009372486996250448 --> [-1, 0, 0, 8.000080000000001e-07, 0, 0, 0, 0] < 0 and [1, 0, 0, -0.0009372486996250448, 0, 0, 0, 0] < 0
fieldTolerance w_leg_factor 4.12525439843647e-05 0.012253019520256961 --> [-1, 0, 0, 0, 4.12525439843647e-05, 0, 0, 0] < 0 and [1, 0, 0, 0, -0.012253019520256961, 0, 0, 0] < 0
fieldTolerance B_design 5.0000500000000003e-05 0.13557761627338663 --> [-1, 0, 0, 0, 0, 5.0000500000000003e-05, 0, 0] < 0 and [1, 0, 0, 0, 0, -0.13557761627338663, 0, 0] < 0
fieldTolerance aper_x 0.00033333666666666674 0.33333000000000007 --> [-1, 0, 0, 0, 0, 0, 0.00033333666666666674, 0] < 0 and [1, 0, 0, 0, 0, 0, -0.33333

In [85]:
def project_to_polytope(x, A_mat=A, b_vec = b):
    # Define decision variable
    p = cp.Variable(len(x))
    # Define objective function
    objective = cp.Minimize(cp.sum_squares(p - x))

    A_mat_dynamic = A_mat[:]
    b_vec_dynamic = b_vec[:]


    # Define constraints
    constraints = [np.array(A_mat) @ p <= np.array(b_vec)]

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()


    # Optimal solution
    closest_point = p.value
    return closest_point

In [94]:
target = torch.tensor([1.8,0.09,0.05], device=device,dtype=torch.double)
loss_fun = torch.nn.MSELoss()

In [95]:
# init guess
guess = torch.randn((8),dtype=torch.double)
guess = network.bn0.running_mean[:8].clone().detach()

# initially project to ensure all values make sense and the continuous transformations make sense  as well
magnet_params = torch.tensor(project_to_polytope(guess.detach().cpu().numpy())).unsqueeze(0).to(device).requires_grad_(True)

lr = 1e-1
lr_tensor = network.bn0.running_mean[:8].clone().detach() * lr
print(lr_tensor.clone().detach())
network.eval()

n_steps = 2000

for step in range(n_steps):
    if magnet_params.grad is not None:
        magnet_params.grad.zero_()
    for layer in network.parameters():
        if layer.grad is not None:
            layer.grad.zero_()

    pred = network(magnet_params)
    #print(pred)
    loss = loss_fun(pred,target)
    #print(loss)

    print(f"{step}, {loss.detach().cpu().numpy():.12f}",end="\r")
    loss.backward()
    with torch.no_grad():        
        magnet_params = magnet_params - lr_tensor * magnet_params.grad
        proj_result = project_to_polytope(magnet_params.squeeze().detach().cpu().numpy())
        if (proj_result is None):
            magnet_params = torch.tensor(magnet_params).requires_grad_(True)
        else:        
            magnet_params = torch.tensor(proj_result).unsqueeze(0).to(device).requires_grad_(True)


magnet_params = torch.tensor(project_to_polytope(magnet_params.squeeze().detach().cpu().numpy())).unsqueeze(0).to(device).requires_grad_(True)
magnet_params



tensor([3.4896e-04, 5.1580e-01, 7.4863e-02, 7.1511e+00, 1.5748e-01, 9.5007e-02,
        1.3921e-02, 8.8204e-03], device='cuda:0', dtype=torch.float64)


/home/max/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor([[9.8067e-03, 5.7211e+00, 9.9266e-01, 9.3665e+01, 1.8459e+00, 2.0000e+00,
         1.5989e-01, 1.5989e-01]], device='cuda:0', dtype=torch.float64,
       requires_grad=True)

In [89]:
import femmcreator
import magnetdesigner

In [91]:
magnet_params_np = magnet_params.squeeze().detach().cpu().numpy()

print(*list(*(magnet_params.detach().cpu().numpy())))
#['fieldTolerance', 'maxCurrentDensity', 'rho0', 'usedPowerInPercent', 'w_leg_factor', 'B_design', 'aper_x', 'aper_y']

magnet = magnetdesigner.designer.get_Dipole(
    name=name, 
    B_design=magnet_params_np[5], 
    aper_x=magnet_params_np[6], 
    aper_y=magnet_params_np[7], 
    fieldTolerance=magnet_params_np[0], 
    rho0=magnet_params_np[2], 
    w_leg_factor=magnet_params_np[4],
    maxCurrentDensity=magnet_params_np[1], 
    usedPowerInPercent=magnet_params_np[3], 
    shape="H")

femm = femmcreator.FEMM()
femm.Build(magnet.get_femminput)
femm.CreateFEMM('dipole_1.8_0.09_0.05')

0.009795387215923832 5.964267567238819 0.9929449227917425 99.73837421491311 1.8656625112835408 1.9999921337070532 0.13590314737957584 0.10416306217856935
